Its a system that uses a webcam to capture a person's facial expression, analyze their emotions, and respond with appropriate visual or textual feedback — like a smart mirror that can "perceive" how you're feeling.

### This project simulates the human perceptual process:
    * Visual sensing (camera as the eye)
    * Interpretation (emotion recognition via machine learning)
    * Emotional response (feedback on screen)

### How it can help:
    * it can help in mental health monitoring
    * Can be used in smart mirrors, educational robots, or autistic care assistants
    * Encourages self-awareness in users by reflecting emotional state
    * Useful for health, education, caregiving, and wellness tech

In [2]:
pip install --user opencv-python fer

  Obtaining dependency information for fer from https://files.pythonhosted.org/packages/19/d7/cc5954aac8db0be7eaff31c5cd059c23af59391d228c9d09cead0250bdae/fer-22.5.1-py3-none-any.whl.metadata
  Using cached fer-22.5.1-py3-none-any.whl.metadata (6.4 kB)
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/7f/8c/ec631100261b0fca25cafd1e1a06592e50b3cda8aa08e7c4c14d7b4d7115/opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_contrib_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Obtaining dependency information for facenet-pytorch from https://files.pythonhosted.org/packages/ed/2e/2d56386bc2f834cdc683743903852cf1428b4e5ee119f16cf808b589d3cd/facenet_pytorch-2.6.0-py3-none-any.whl.metadata
  Using cached facenet_pytorch-2.6.0-py3-none-any.whl.metadata (12 kB)
  Obtaining dependency information for moviepy from https://files.pythonhosted.org/packages/9a/73/7d3b2010baa0b5eb1e4dfa9e4385e89b6716b

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2 requires torch==2.0.1, but you have torch 2.2.2 which is incompatible.


In [1]:
import cv2
from fer import FER
import time
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# === Emotion Logger Function ===
def log_emotion(emotion):
    df = pd.DataFrame([[datetime.now(), emotion]], columns=["timestamp", "emotion"])
    df.to_csv("emotion_log.csv", mode='a', header=False, index=False)
    print(f"Emotion logged: {emotion}")


# Initialize the webcam and emotion detector
cap = cv2.VideoCapture(0)  # 0 = default webcam
emotion_detector = FER(mtcnn=True)

print("Starting Emotion-Aware Mirror (Press 'q' to quit)...")

# === Cooldown setup ===
last_logged_time = 0
cooldown = 5  # seconds


while True:
    ret, frame = cap.read()
    if not ret:
        print("Could not read frame from camera")
        break
    
     # Detect emotion in the frame
    emotion_results = emotion_detector.detect_emotions(frame)

    for result in emotion_results:
        (x, y, w, h) = result["box"]
        emotions = result["emotions"]
        top_emotion = max(emotions, key=emotions.get)

        # Draw bounding box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display the top emotion
        label = f"{top_emotion}: {emotions[top_emotion]*100:.1f}%"
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Smart response (perceptual feedback)
        if top_emotion == "happy":
            response = "You look great today!"
        elif top_emotion == "sad":
            response = "Stay strong. You're doing your best!"
        elif top_emotion == "angry":
            response = "Take a deep breath. You've got this."
        elif top_emotion == "surprise":
            response = "Something exciting happening?"
        else:
            response = "Hope you're having a good day."

        # Show response on screen
        cv2.putText(frame, response, (10, 30),
                    cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 0, 255), 2)
        
        # Log the emotion with cooldown
        current_time = time.time()
        if current_time - last_logged_time > cooldown:
            log_emotion(top_emotion)
            last_logged_time = current_time

    # Display the frame
    cv2.imshow("Emotion-Aware Mirror", frame)

    # Break on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

Starting Emotion-Aware Mirror (Press 'q' to quit)...
Emotion logged: neutral
Emotion logged: neutral
Emotion logged: neutral
Emotion logged: sad
Emotion logged: angry
Emotion logged: sad
Emotion logged: neutral
Emotion logged: neutral
Emotion logged: neutral


### Log Emotions and Give Feedback

In [4]:
emotion_detected = top_emotion

'''
# Assume this is the emotion detected
emotion_detected = "angry"

# Save to CSV log
def log_emotion(emotion):
    df = pd.DataFrame([[datetime.now(), emotion]], columns=["timestamp", "emotion"])
    df.to_csv("emotion_log.csv", mode='a', header=False, index=False)
    print(f"Emotion logged: {emotion}")

'''
# Respond with wellness suggestion
def suggest_wellness(emotion):
    suggestions = {
        "happy": "Keep up the positive energy!",
        "sad": "Try a 5-minute meditation or talk to someone.",
        "angry": "Take a break, deep breaths help.",
        "surprise": "Something unexpected? Stay grounded.",
        "neutral": " You seem calm. Good time for focus.",
        "fear": "You may be anxious. Try calming music.",
        "disgust": "Step away from unpleasant things if possible."
    }
    print(f"Suggestion: {suggestions.get(emotion, 'Stay strong and be kind to yourself.')}")

In [5]:
# Visualize emotion trends
def plot_emotion_trend():
    try:
        df = pd.read_csv("emotion_log.csv", names=["timestamp", "emotion"], parse_dates=["timestamp"])
        df["date"] = df["timestamp"].dt.date
        trend = df.groupby(["date", "emotion"]).size().unstack().fillna(0)

        trend.plot(kind="bar", stacked=True, figsize=(10,5))
        plt.title("Daily Emotion Trends")
        plt.xlabel("Date")
        plt.ylabel("Emotion Count")
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print("Could not generate chart:", e)

In [6]:
# Use it
log_emotion(emotion_detected)
suggest_wellness(emotion_detected)
plot_emotion_trend()

Emotion logged: neutral
Suggestion: You seem calm. Good time for focus.
Could not generate chart: Can only use .dt accessor with datetimelike values


### Launch a Meditation or Sound Clip

In [7]:
import webbrowser

def open_meditation():
    # Open a calming YouTube video
    webbrowser.open("https://www.youtube.com/watch?v=inpok4MKVLM")  # 5-min meditation

if emotion_detected in ["sad", "angry", "fear", "neutral"]:
    open_meditation()